# זרימת עבודה עם אדם בתהליך באמצעות Microsoft Agent Framework

## 🎯 מטרות למידה

במחברת זו תלמדו כיצד ליישם זרימות עבודה עם **אדם בתהליך** באמצעות `RequestInfoExecutor` של Microsoft Agent Framework. תבנית עוצמתית זו מאפשרת להשהות זרימות עבודה של AI כדי לאסוף קלט אנושי, מה שהופך את הסוכנים לאינטראקטיביים ומעניק לבני אדם שליטה על החלטות קריטיות.

## 🔄 מהו אדם בתהליך?

**אדם בתהליך (HITL)** הוא תבנית עיצוב שבה סוכני AI עוצרים את הביצוע כדי לבקש קלט אנושי לפני שהם ממשיכים. זה חיוני עבור:

- ✅ **החלטות קריטיות** - קבלת אישור אנושי לפני ביצוע פעולות חשובות
- ✅ **מצבים מעורפלים** - מתן אפשרות לבני אדם להבהיר כאשר AI אינו בטוח
- ✅ **העדפות משתמש** - בקשת בחירה ממשתמשים בין מספר אפשרויות
- ✅ **ציות ובטיחות** - הבטחת פיקוח אנושי על פעולות מוסדרות
- ✅ **חוויות אינטראקטיביות** - יצירת סוכנים שיחה שמגיבים לקלט משתמש

## 🏗️ איך זה עובד ב-Microsoft Agent Framework

המסגרת מספקת שלושה רכיבים מרכזיים עבור HITL:

1. **`RequestInfoExecutor`** - מבצע מיוחד שמשהה את זרימת העבודה ומשדר אירוע `RequestInfoEvent`
2. **`RequestInfoMessage`** - מחלקת בסיס עבור מטעני בקשה מוקלדים שנשלחים לבני אדם
3. **`RequestResponse`** - מקשר בין תגובות אנושיות לבקשות המקוריות באמצעות `request_id`

**תבנית זרימת עבודה:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 הדוגמה שלנו: הזמנת מלון עם אישור משתמש

נבנה על זרימת העבודה המותנית על ידי הוספת אישור אנושי **לפני** הצעת יעדים חלופיים:

1. המשתמש מבקש יעד (לדוגמה, "פריז")
2. `availability_agent` בודק אם יש חדרים זמינים
3. **אם אין חדרים** → `confirmation_agent` שואל "האם תרצה לראות חלופות?"
4. זרימת העבודה **מושהית** באמצעות `RequestInfoExecutor`
5. **המשתמש מגיב** "כן" או "לא" דרך קלט בקונסולה
6. `decision_manager` מנתב על סמך התגובה:
   - **כן** → הצגת יעדים חלופיים
   - **לא** → ביטול בקשת ההזמנה
7. הצגת התוצאה הסופית

זה מדגים כיצד לתת למשתמשים שליטה על ההצעות של הסוכן!

---

בואו נתחיל! 🚀


## שלב 1: ייבוא ספריות נדרשות

אנו מייבאים את רכיבי מסגרת הסוכן הסטנדרטיים בנוסף ל**מחלקות ספציפיות למעורבות אנושית**:
- `RequestInfoExecutor` - מבצע שמפסיק את זרימת העבודה לצורך קלט אנושי
- `RequestInfoEvent` - אירוע שנפלט כאשר נדרש קלט אנושי
- `RequestInfoMessage` - מחלקת בסיס עבור מטעני בקשה מסוגים שונים
- `RequestResponse` - מקשר בין תגובות אנושיות לבקשות
- `WorkflowOutputEvent` - אירוע לזיהוי פלטים של זרימת עבודה


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## שלב 2: הגדרת מודלים של Pydantic עבור פלטים מובנים

מודלים אלו מגדירים את **הסכמה** שהסוכנים יחזירו. אנו שומרים את כל המודלים מזרימת העבודה המותנית ומוסיפים:

**חדש עבור מעורבות אנושית:**
- `HumanFeedbackRequest` - תת-מחלקה של `RequestInfoMessage` שמגדירה את מטען הבקשה שנשלח לבני אדם  
  - מכיל `prompt` (שאלה לשאול) ו-`destination` (הקשר לגבי העיר שאינה זמינה)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## שלב 3: יצירת כלי להזמנת מלון

אותו כלי מהתהליך המותנה - בודק אם יש חדרים זמינים ביעד.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## שלב 4: הגדרת פונקציות תנאי לניתוב

אנחנו צריכים **ארבע פונקציות תנאי** עבור זרימת העבודה שלנו עם מעורבות אנושית:

**מזרימת עבודה מותנית:**
1. `has_availability_condition` - מנתב כאשר יש זמינות בבתי מלון
2. `no_availability_condition` - מנתב כאשר אין זמינות בבתי מלון

**חדש עבור מעורבות אנושית:**
3. `user_wants_alternatives_condition` - מנתב כאשר המשתמש אומר "כן" לחלופות
4. `user_declines_alternatives_condition` - מנתב כאשר המשתמש אומר "לא" לחלופות


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## שלב 5: יצירת מנהל ההחלטות המבצע

זהו **הליבה של תבנית האדם-במעגל**! ה-`DecisionManager` הוא `Executor` מותאם אישית שמבצע את הפעולות הבאות:

1. **מקבל משוב אנושי** באמצעות אובייקטים מסוג `RequestResponse`
2. **מעבד את החלטת המשתמש** (כן/לא)
3. **מנתב את זרימת העבודה** על ידי שליחת הודעות לסוכנים המתאימים

תכונות עיקריות:
- משתמש במעטר `@handler` כדי לחשוף שיטות כשלבים בזרימת העבודה
- מקבל `RequestResponse[HumanFeedbackRequest, str]` המכיל גם את הבקשה המקורית וגם את תשובת המשתמש
- מניב הודעות פשוטות של "כן" או "לא" שמפעילות את פונקציות התנאי שלנו


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## שלב 6: יצירת מבצע תצוגה מותאם אישית

אותו מבצע תצוגה מתוך זרימת עבודה מותנית - מספק תוצאות סופיות כפלט זרימת העבודה.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## שלב 7: טעינת משתני סביבה

הגדר את לקוח ה-LLM (GitHub Models, Azure OpenAI, או OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## שלב 8: יצירת סוכני AI ומבצעים

אנו יוצרים **שישה רכיבי זרימת עבודה**:

**סוכנים (עטופים ב-AgentExecutor):**
1. **availability_agent** - בודק זמינות מלונות באמצעות הכלי
2. **confirmation_agent** - 🆕 מכין בקשת אישור מהאדם
3. **alternative_agent** - מציע ערים חלופיות (כאשר המשתמש אומר כן)
4. **booking_agent** - מעודד הזמנה (כאשר יש חדרים זמינים)
5. **cancellation_agent** - 🆕 מטפל בהודעת ביטול (כאשר המשתמש אומר לא)

**מבצעים מיוחדים:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` שעוצר את זרימת העבודה לצורך קלט אנושי
7. **decision_manager** - 🆕 מבצע מותאם אישית שמנתב לפי תגובת האדם (כבר הוגדר לעיל)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## שלב 9: בניית זרימת העבודה עם מעורבות אנושית

כעת אנו בונים את גרף זרימת העבודה עם **ניתוב מותנה** הכולל את המסלול עם מעורבות אנושית:

**מבנה זרימת העבודה:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**קצוות מרכזיים:**
- `availability_agent → confirmation_agent` (כאשר אין חדרים)
- `confirmation_agent → prepare_human_request` (שינוי סוג)
- `prepare_human_request → request_info_executor` (הפסקה למעורבות אנושית)
- `request_info_executor → decision_manager` (תמיד - מספק RequestResponse)
- `decision_manager → alternative_agent` (כאשר המשתמש אומר "כן")
- `decision_manager → cancellation_agent` (כאשר המשתמש אומר "לא")
- `availability_agent → booking_agent` (כאשר יש חדרים זמינים)
- כל המסלולים מסתיימים ב-`display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## שלב 10: הרצת מקרה בדיקה 1 - עיר ללא זמינות (פריז עם אישור אנושי)

מקרה בדיקה זה מדגים את **המעגל המלא של מעורבות אנושית**:

1. בקשת מלון בפריז
2. availability_agent בודק → אין חדרים
3. confirmation_agent יוצר שאלה המיועדת לאדם
4. request_info_executor **עוצר את זרימת העבודה** ומשדר `RequestInfoEvent`
5. **האפליקציה מזהה את האירוע ומבקשת תשובה מהמשתמש בקונסולה**
6. המשתמש מקליד "כן" או "לא"
7. האפליקציה שולחת את התשובה דרך `send_responses_streaming()`
8. decision_manager מנתב בהתאם לתשובה
9. התוצאה הסופית מוצגת

**תבנית מרכזית:**
- השתמש ב-`workflow.run_stream()` עבור האיטרציה הראשונה
- השתמש ב-`workflow.send_responses_streaming(pending_responses)` עבור האיטרציות הבאות
- האזן ל-`RequestInfoEvent` כדי לזהות מתי נדרש קלט אנושי
- האזן ל-`WorkflowOutputEvent` כדי ללכוד את התוצאות הסופיות


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## שלב 11: הרצת מקרה בדיקה 2 - עיר עם זמינות (סטוקהולם - ללא צורך בקלט אנושי)

בדיקה זו מדגימה את **המסלול הישיר** כאשר יש חדרים זמינים:

1. בקשת מלון בסטוקהולם  
2. availability_agent בודק → חדרים זמינים ✅  
3. booking_agent מציע הזמנה  
4. display_result מציג אישור  
5. **אין צורך בקלט אנושי!**

תהליך העבודה עוקף לחלוטין את המסלול הכולל מעורבות אנושית כאשר יש חדרים זמינים.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## נקודות מפתח ופרקטיקות מומלצות לשילוב האדם בתהליך

### ✅ מה למדתם:

#### 1. **תבנית RequestInfoExecutor**
התבנית לשילוב האדם בתהליך במסגרת Microsoft Agent Framework משתמשת בשלושה רכיבים מרכזיים:
- `RequestInfoExecutor` - עוצר את זרימת העבודה ומשדר אירועים
- `RequestInfoMessage` - מחלקת בסיס למטעני בקשה מסוגים שונים (יש להרחיב אותה!)
- `RequestResponse` - מקשר בין תגובות אנושיות לבקשות המקוריות

**הבנה קריטית:**
- `RequestInfoExecutor` אינו אוסף קלט בעצמו - הוא רק עוצר את זרימת העבודה
- קוד האפליקציה שלכם חייב להאזין ל-`RequestInfoEvent` ולאסוף קלט
- עליכם לקרוא ל-`send_responses_streaming()` עם מילון שממפה `request_id` לתשובת המשתמש

#### 2. **תבנית ביצוע בסטרימינג**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **ארכיטקטורה מבוססת אירועים**
האזינו לאירועים ספציפיים כדי לשלוט בזרימת העבודה:
- `RequestInfoEvent` - נדרש קלט אנושי (זרימת העבודה נעצרת)
- `WorkflowOutputEvent` - התוצאה הסופית זמינה (זרימת העבודה הושלמה)
- `WorkflowStatusEvent` - שינויי מצב (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, וכו')

#### 4. **מבצעי מותאמים אישית עם @handler**
ה-`DecisionManager` מדגים כיצד ליצור מבצעים שמבצעים:
- שימוש במעטר `@handler` כדי לחשוף שיטות כשלבים בזרימת העבודה
- קבלת הודעות מסוגים שונים (לדוגמה, `RequestResponse[HumanFeedbackRequest, str]`)
- ניתוב זרימת העבודה על ידי שליחת הודעות למבצעים אחרים
- גישה להקשר דרך `WorkflowContext`

#### 5. **ניתוב מותנה עם החלטות אנושיות**
ניתן ליצור פונקציות תנאי שמעריכות תגובות אנושיות:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 יישומים בעולם האמיתי:

1. **זרימות עבודה לאישור**
   - קבלת אישור מנהל לפני עיבוד דוחות הוצאות
   - דרישת סקירה אנושית לפני שליחת מיילים אוטומטיים
   - אישור עסקאות בעלות ערך גבוה לפני ביצוע

2. **מתן תוכן**
   - סימון תוכן בעייתי לסקירה אנושית
   - בקשת החלטה סופית ממנחים במקרים גבוליים
   - העברת המקרה לאדם כאשר הביטחון של AI נמוך

3. **שירות לקוחות**
   - מתן מענה אוטומטי לשאלות שגרתיות
   - העברת בעיות מורכבות לנציגים אנושיים
   - שאלת הלקוח אם הוא מעוניין לדבר עם אדם

4. **עיבוד נתונים**
   - בקשת פתרון אנושי לנתונים מעורפלים
   - אישור פרשנויות AI של מסמכים לא ברורים
   - מתן אפשרות למשתמשים לבחור בין פרשנויות תקפות שונות

5. **מערכות קריטיות לבטיחות**
   - דרישת אישור אנושי לפני פעולות בלתי הפיכות
   - קבלת אישור לפני גישה לנתונים רגישים
   - אישור החלטות בתעשיות מוסדרות (בריאות, פיננסים)

6. **סוכנים אינטראקטיביים**
   - בניית בוטים שיחתיים ששואלים שאלות המשך
   - יצירת אשפים שמנחים משתמשים בתהליכים מורכבים
   - עיצוב סוכנים שמשתפים פעולה עם בני אדם שלב אחר שלב

### 🔄 השוואה: עם מול בלי שילוב האדם בתהליך

| תכונה | זרימת עבודה מותנית | זרימת עבודה עם שילוב האדם בתהליך |
|-------|---------------------|----------------------------------|
| **ביצוע** | `workflow.run()` יחיד | לולאה עם `run_stream()` + `send_responses_streaming()` |
| **קלט משתמש** | אין (אוטומטי לחלוטין) | הנחיות אינטראקטיביות דרך `input()` או ממשק משתמש |
| **רכיבים** | סוכנים + מבצעים | + RequestInfoExecutor + DecisionManager |
| **אירועים** | רק AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, וכו' |
| **עצירה** | אין עצירה | זרימת העבודה נעצרת ב-RequestInfoExecutor |
| **שליטה אנושית** | אין שליטה אנושית | בני אדם מקבלים החלטות מרכזיות |
| **שימוש** | אוטומציה | שיתוף פעולה ופיקוח |

### 🚀 תבניות מתקדמות:

#### נקודות החלטה אנושיות מרובות
ניתן להוסיף מספר צמתים של `RequestInfoExecutor` באותה זרימת עבודה:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### טיפול בזמני קצוב
יישום זמני קצוב לתגובות אנושיות:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### שילוב ממשק משתמש עשיר
במקום `input()`, שילוב עם ממשק משתמש אינטרנטי, Slack, Teams, וכו':
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### שילוב האדם בתהליך מותנה
בקשת קלט אנושי רק במצבים ספציפיים:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ פרקטיקות מומלצות:

1. **תמיד הרחיבו את RequestInfoMessage**
   - מספק בטיחות סוגים ואימות
   - מאפשר הקשר עשיר לעיצוב ממשק משתמש
   - מבהיר את כוונת כל סוג בקשה

2. **השתמשו בהנחיות תיאוריות**
   - כללו הקשר לגבי מה שאתם שואלים
   - הסבירו את ההשלכות של כל בחירה
   - שמרו על שאלות פשוטות וברורות

3. **טפלו בקלט בלתי צפוי**
   - אימות תגובות משתמש
   - מתן ברירות מחדל לקלט לא תקין
   - מתן הודעות שגיאה ברורות

4. **עקבו אחר מזהי בקשות**
   - השתמשו בקשר בין request_id לתגובות
   - אל תנסו לנהל מצב באופן ידני

5. **עצבו ללא חסימה**
   - אל תחסמו תהליכים בהמתנה לקלט
   - השתמשו בתבניות אסינכרוניות לאורך כל הדרך
   - תמכו במופעים מקבילים של זרימת עבודה

### 📚 מושגים קשורים:

- **Agent Middleware** - יירוט קריאות סוכן (מחברת קודמת)
- **ניהול מצב זרימת עבודה** - שמירת מצב זרימת עבודה בין ריצות
- **שיתוף פעולה בין סוכנים** - שילוב האדם בתהליך עם צוותי סוכנים
- **ארכיטקטורות מבוססות אירועים** - בניית מערכות תגובתיות עם אירועים

---

### 🎓 ברכות!

שלטתם בזרימות עבודה עם שילוב האדם בתהליך במסגרת Microsoft Agent Framework! עכשיו אתם יודעים:
- ✅ לעצור זרימות עבודה כדי לאסוף קלט אנושי
- ✅ להשתמש ב-RequestInfoExecutor וב-RequestInfoMessage
- ✅ לטפל בביצוע בסטרימינג עם אירועים
- ✅ ליצור מבצעים מותאמים אישית עם @handler
- ✅ לנתב זרימות עבודה על בסיס החלטות אנושיות
- ✅ לבנות סוכנים אינטראקטיביים שמשתפים פעולה עם בני אדם

**זו תבנית קריטית לבניית מערכות AI אמינות ונשלטות!** 🚀



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום AI [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש להיות מודעים לכך שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי אנושי. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
